# SoH estimation from estimated range

## Setup

### Imports

### Data extraction

In [ ]:
import pandas as pd
from core.sql_utils import get_connection
import numpy as np
from core.plt_utils import plt_3d_df
import plotly.express as px
from core.s3.s3_utils import S3Service
from core.s3.settings import S3Settings
from core.spark_utils import create_spark_session
from sqlalchemy import text
import pyspark.sql.functions as F

settings = S3Settings()

spark = create_spark_session(
    settings.S3_KEY,
    settings.S3_SECRET
)

s3 = S3Service()

In [ ]:
raw_tss = s3.read_parquet_df_spark(spark, "raw_ts/volvo-cars/time_series/raw_ts_spark.parquet").toPandas()
raw_tss[["average_speed","battery_level","distance_since_reset","electric_consumption_average","estimated_range","odometer","time_to_complete_charge"]] = raw_tss[["average_speed","battery_level","distance_since_reset","electric_consumption_average","estimated_range","odometer","time_to_complete_charge"]].astype(float)

In [ ]:
raw_tss_to_plot = (
    raw_tss
    .eval("distance_since_reset = distance_since_reset.ffill()")
    .eval("average_speed = average_speed.ffill()")
)

In [ ]:
px.scatter(
    raw_tss_to_plot,
    x="distance_since_reset",
    y="electric_consumption_average",
    color="vin"
).show()

In [ ]:

plt_3d_df(
    raw_tss_to_plot.query("electric_consumption_average < 50 & electric_consumption_average > 10 & distance_since_reset > 100 & average_speed > 0"),
    x="distance_since_reset",
    y="average_speed",
    z="electric_consumption_average",
    color="vin"
)